In [2]:
import requests 
import pandas as pd    
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os


In [3]:
load_dotenv(dotenv_path='../erdos_finance_llm\erdos.env')

def get_news(symbols, start_date, end_date,api_key):
    if api_key is None:
        api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
        if api_key is None:
            raise ValueError("API key not found in environment variables")
    if start_date is None:
        start_date = (datetime.now() - timedelta(days=7)).strftime('%Y%m%dT0000')
    else:
        start_date = datetime.strptime(start_date, '%Y-%m-%d').strftime('%Y%m%dT0000')
    if end_date is None:
        end_date = datetime.now().strftime('%Y%m%dT0000')
    else:
        end_date = datetime.strptime(end_date, '%Y-%m-%d').strftime('%Y%m%dT0000')

    news_data = []
    try:
        for symbol in symbols:
            url = f"https://www.alphavantage.co/query"
            params ={
                'function': 'NEWS_SENTIMENT',
                'tickers': symbol,
                'apikey': api_key,
                'time_from': start_date,
                'time_to': end_date,
                #'sort': 'LATEST',
                
            }
            response = requests.get(url, params=params)
            data = response.json()
            if 'feed' not in data:
                print(f"No news found for {symbol} in the given date range")
                continue
            for item in data['feed']:
               news = {
                   'symbol': symbol,
                   'title': item['title'],
                   'source': item['source'],
                   'date': datetime.strptime(item['time_published'], '%Y%m%dT%H%M%S'),
                   'summary': item['summary'],
                   'url': item['url'],
               }
               
               if 'ticker_sentiment' in item:
                   for ticker_data in item['ticker_sentiment']:
                       if ticker_data['ticker'] == symbol:
                           news['sentiment'] = ticker_data['ticker_sentiment_label']
                           news['sentiment_score'] = ticker_data['ticker_sentiment_score']
                           break

               news_data.append(news)
                
    except Exception as e:
        print(f"Error fetching news for {symbol}: {e}")
        return pd.DataFrame()
    
    if news_data:
        news_df = pd.DataFrame(news_data)
        news_df = news_df.sort_values(by='date', ascending=False)
        return news_df
    else:
        return pd.DataFrame()

    

                        


<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\zxmat\AppData\Local\Temp\ipykernel_42016\1104685585.py:1: SyntaxWarning: invalid escape sequence '\e'
  load_dotenv(dotenv_path='../erdos_finance_llm\erdos.env')


In [24]:
test_news = get_news(
    symbols=["TSLA"],
    start_date="2025-03-01",
    end_date="2025-03-05",
    api_key=os.getenv('ALPHA_VANTAGE_API_KEY')
)

In [25]:
test_news

,symbol,title,source,date,summary,url,sentiment,sentiment_score
0,TSLA,Elon Musk Offered A Cybertruck Bounty To Unmas...,Benzinga,2025-03-04 23:57:23,Sen. Cory Booker ( D-N.J. ) confessed to spear...,https://www.benzinga.com/25/03/44131027/elon-m...,Neutral,-0.030161
1,TSLA,"Tesla Sales Drop In Europe In February, Some C...",Benzinga,2025-03-04 22:06:36,Tesla sales fell in the month of February in s...,https://www.benzinga.com/news/global/25/03/441...,Neutral,-0.039422
2,TSLA,February's 20 Most-Searched Tickers On Pro - W...,Benzinga,2025-03-04 20:36:44,A look at the top 20 most-searched tickers on ...,https://www.benzinga.com/trading-ideas/25/03/4...,Somewhat-Bullish,0.203056
3,TSLA,"Nvidia, Apple, and Eli Lilly: Manufacturing th...",Motley Fool,2025-03-04 18:36:00,The great buildout for AI and GLP-1 drugs cont...,https://www.fool.com/investing/2025/03/04/nvid...,Neutral,0.03488
4,TSLA,Bitcoin price stabilizes near $83K as investor...,Cointelegraph,2025-03-04 17:49:09,"Bitcoin price volatility continues, but BTC de...",https://cointelegraph.com/news/bitcoin-price-s...,Neutral,0.067024
5,TSLA,Time to Buy the Dip? 3 Rare Signals Appear,Zacks Commentary,2025-03-04 17:32:00,Market corrections are inevitable and are ofte...,https://www.zacks.com/commentary/2425405/time-...,Somewhat-Bearish,-0.229448
6,TSLA,Tesla faces 'slower growth and rising risk' am...,MarketWatch,2025-03-04 17:04:00,BofA Securities has cut its price target on Te...,https://www.marketwatch.com/story/tesla-faces-...,Somewhat-Bearish,-0.221282
7,TSLA,Investors Shouldn't 'Run To The Hills' Over Ta...,Benzinga,2025-03-04 16:53:01,Technology stocks are falling Tuesday on fears...,https://www.benzinga.com/analyst-ratings/analy...,Neutral,0.0
8,TSLA,"Raising Cash And Hedges, Pay Attention To Stoc...",Benzinga,2025-03-04 16:33:34,"To gain an edge, this is what you need to know...",https://www.benzinga.com/markets/equities/25/0...,Neutral,-0.140649
9,TSLA,Sirius XM Plunges 42.3% in a Year: How Should ...,Zacks Commentary,2025-03-04 16:17:00,"SIRI expands content and improves engagement, ...",https://www.zacks.com/stock/news/2425299/siriu...,Neutral,0.121423


In [21]:
NEWS_ANALYSIS_PROMPT = """
    You are an expert financial analyst.
    I will provide you with a news article related to a stock.
    1. Determine sentiment: Positive, Negative, Neutral.
    2. Summary key points relevant to the stock.
    3. Provide an investment recommendation based on this specific article.
    
    Format your output as :
    {
        "sentiment": "Positive/Negative/Neutral",
        "key_points": [
            "...",
            "..."
        ],
        # "summary": "...",
        "investment_recommendation": "..."
    }
    """

In [20]:
from groq import Groq
import json 
import re

In [22]:
def analyze_news(news_df, api_key=None, model='llama3-8b-8192'):
    if api_key is None:
        print('Api_key None')
    client = Groq(api_key=api_key)
    
    news_df['date_only'] = news_df['date'].dt.date
    
    grouped_news = news_df.groupby(['symbol', 'date_only'])
    
    results = []
    
    for (symbol, date), group in grouped_news:
        print(f"Processing {symbol} on {date}")
        articles_text = '\n\n'.join([
            f'Title: {row["title"]}\nContent: {row["summary"]}'
            for _, row in group.iterrows()]
        )
        try:
            completion = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": NEWS_ANALYSIS_PROMPT},
                    {"role": "user", "content": f"Stock: {symbol}\nDate: {date}\nNews Articles:\n{articles_text}"}
                ],
                model = model,
                temperature=0.1
            )
            analysis = completion.choices[0].message.content
            
            try:
                result = json.loads(analysis)
            except:
               
                
                cleaned_text = analysis.strip()
                
                if '{' in cleaned_text:
                    cleaned_text = cleaned_text[cleaned_text.find('{'):]
                
                if '}' in cleaned_text:
                    cleaned_text = cleaned_text[:cleaned_text.rfind('}')+1]
                
                try:
                    result = json.loads(cleaned_text)
                except:
                    print(f"Failed to parse JSON for {symbol} on {date}")
                    print(f"Raw response: {analysis}...") 
                    result = {
                        "sentiment": "parsing_error",
                        "key_points": ['parsing_error'],
                        "investment_recommendation": "failed"
                    }
            results.append({
                'symbol': symbol,
                'date': date,
                'news_count': len(group),
                'ai_sentiment': result.get('sentiment', ''),
                'ai_key_points': result.get('key_points', []),
                'ai_recommendation': result.get('investment_recommendation', '')
            })
        except Exception as e:
            print(f"Error processing {symbol} on {date}: {e}")
            results.append({
                'symbol': symbol,
                'date': date,
                'news_count': len(group),
                'ai_sentiment': 'processing_error',
                'ai_key_points': str([f'processing_error:{str(e)}']),
                'ai_recommendation': f'processing_error:{str(e)}'
                
                
            })
                
    return pd.DataFrame(results)


In [26]:
analyze_news(test_news, api_key=os.getenv('GROQ_API_KEY'), model='llama3-8b-8192')

Processing TSLA on 2025-03-03
Processing TSLA on 2025-03-04
Failed to parse JSON for TSLA on 2025-03-04
Raw response: Here is the analysis of the news articles related to TSLA:

**Sentiment:** Neutral

**Key Points:**

1. Elon Musk offered a Cybertruck bounty to unmask a Trump social media critic, which was responded to by Sen. Cory Booker.
2. Tesla sales dropped in Europe in February, with some countries experiencing a decline of over 40%.
3. The company's focus on artificial intelligence and robotics is seen as a positive by Morgan Stanley, which reinstated Tesla as its top U.S. auto pick.
4. Tesla introduced 0% APR financing on Model 3 purchases amid falling demand.
5. The company's Chair, Robyn Denholm, sold over $30 million worth of stock as the automaker grapples with macroeconomic uncertainties and increased competitive heat.

**Investment Recommendation:** Hold

The sentiment of the news articles is neutral, with some positive and negative points. While the company's focus on A

,symbol,date,news_count,ai_sentiment,ai_key_points,ai_recommendation
0,TSLA,2025-03-03,13,Negative,[Uber CEO Dara Khosrowshahi warns Tesla invest...,Based on the negative sentiment and challenges...
1,TSLA,2025-03-04,37,parsing_error,[parsing_error],failed


In [30]:
output = analyze_news(test_news, api_key=os.getenv('GROQ_API_KEY'), model='llama3-8b-8192')


Processing TSLA on 2025-03-03
Processing TSLA on 2025-03-04
Failed to parse JSON for TSLA on 2025-03-04
Raw response: Here is the analysis of the news articles related to TSLA:

**Sentiment:** Negative

**Key Points:**

* Tesla sales drop in Europe in February, with some countries experiencing a decline of over 40%
* Elon Musk's political push may be hurting the company's sales
* Tesla introduces 0% APR financing on Model 3 purchases amid falling demand
* Morgan Stanley reinstates Tesla as its top U.S. auto pick, citing the company's focus on artificial intelligence and robotics
* Tesla Chair Robyn Denholm sells over $30 million worth of stock as the company grapples with macroeconomic uncertainties and increased competitive heat

**Investment Recommendation:**

Based on the news articles, I would recommend a cautious approach to investing in TSLA. The company is facing challenges in Europe, and its sales are declining. Additionally, the company's political push may be hurting its sale

In [32]:
print(output['ai_key_points'][0])

['Uber CEO warns Tesla investors about ride-sharing challenges', 'Polestar offers promotions to lure Tesla customers away', "Tesla shares decline due to Trump's tariff plans", 'Tesla stock tumbled 28% in February due to various factors', 'Berkshire Hathaway surpasses Tesla in market value', 'Analysts warn of an overdue S&P 500 correction']
